In [160]:
import numpy as np
import pandas as pd
import pandas_profiling
import os
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

#from matplotlib import scatter_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import Lasso, Ridge, LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from zipfile import ZipFile

In [134]:
# import and clean test data

df = pd.read_pickle('pickle_copy.pkl')
df.at[7655,'REGION'] = 5

In [154]:
# print(df.shape)
# print(df.index.value_counts())
df.sort_index()

,INSTNM,HIGHDEG,CONTROL,REGION,LOCALE,CURROPER,TUITFTE,INEXPFTE,CDR3,LO_INC_DEBT_N,MD_INC_DEBT_N,HI_INC_DEBT_N,ICLEVEL,year
0,Community College of the Air Force,2,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1996
0,Alabama A & M University,4,1.0,5.0,NaN,NaN,9590.0,5888.0,0.189,1940,834,311,1.0,2016
0,Alabama A & M University,4,1.0,5.0,NaN,NaN,2998.0,3212.0,NaN,1147,196,36,1.0,1998
0,Alabama A & M University,4,1.0,5.0,12.0,1.0,9227.0,7298.0,0.198,NaN,NaN,NaN,1.0,2017
0,Alabama A & M University,4,1.0,5.0,NaN,NaN,3486.0,2889.0,NaN,1610,421,133,1.0,2000
0,Alabama A & M University,4,1.0,5.0,NaN,NaN,5509.0,5619.0,NaN,1543,749,293,1.0,2009
0,Alabama A & M University,4,1.0,5.0,NaN,NaN,6746.0,6601.0,NaN,1877,905,345,1.0,2010
0,Community College of the Air Force,2,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1997
0,Alabama A & M University,4,1.0,5.0,NaN,NaN,9657.0,7941.0,0.165,1940,835,319,1.0,2015
0,Alabama A & M University,4,1.0,5.0,NaN,NaN,3519.0,3100.0,NaN,1671,528,173,1.0,2001


In [67]:
df.dropna(subset=['CDR3', 'CONTROL'], inplace=True)
df = df.replace('PrivacySuppressed', np.NaN)
df.dropna(subset=['LO_INC_DEBT_N', 'MD_INC_DEBT_N', 'HI_INC_DEBT_N'], inplace=True)
df['Public_or_Private'] = df.CONTROL.map({2:0, 1:1, 3:1})

In [79]:
def create_logistic_regression(df, predictor_columns, predicted_column):
    """
    Creates a logistic regression from input column names to predictor column
    
    Parameters
    ----------
    df: pandas dataframe
        the dataframe's columns should include the predictor_columns and predicted_column
    
    predictor_columns: list
        Should be a subset of columns from df. 
        Should have empty intersection with predicted_column
    
    predicted_column: string
        Should be an element in the list of columns from df. 
        Should not be included in predictor_columns
        
    Returns
    -------
    logreg: logistic regression already trained on training data from predictor columns
    
    Note: This function will change the given dataframe
    """
    
    # Create dataframes( or series) with predictors and  predicted values
    X = df[predictor_columns]
    y = df[predicted_column]
    
    # Scale the data using Robust Scaler
    scale = RobustScaler()
    transformed = scale.fit_transform(X)
    X = pd.DataFrame(transformed, columns = X.columns)
    
    # Create Train and Test Data
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    
    # Create a logistic regression model
    logreg = LogisticRegression(fit_intercept=False, C=1e12, solver='lbfgs')
    
    # Fit the model to the training data
    try:
        model_log = logreg.fit(X_train, y_train)
    except:
        model_log = logreg.fit(np.array(X_train).reshape(-1,1), y_train)
        
    # Add new columns to the given data frame with predicted values and probability of correct predictions
    try:
        df['Predicted_'+predicted_column] = logreg.predict(X)
        df['ProbCorrect_Predicted_'+predicted_column] = logreg.predict_proba(X)[:,1]
    except:
        df['Predicted_'+predicted_column] = logreg.predict(np.array(X).reshape(-1,1))
        df['ProbCorrect_Predicted_'+predicted_column] = logreg.predict_proba(np.array(X).reshape(-1,1))[:,1]
    
    return logreg

In [80]:
# Test create_logistic_regression

create_logistic_regression(df, ['CDR3', 'HI_INC_DEBT_N'], 'Public_or_Private'
                          ).score(df[['CDR3', 'HI_INC_DEBT_N']], df['Public_or_Private'])

0.7426352841647279

In [135]:
def back_fill_from_year(df, year):
    fill_values = {np.NaN: np.NaN}
    for name in df.loc[df.year==year].INSTNM:
        fill_values[name] = {'LOCALE': df.loc[(df.INSTNM==name)&(df.year==year)].LOCALE.values[0], 
                             'CURROPER': df.loc[(df.INSTNM==name)&(df.year==year)].CURROPER.values[0]}
    
    for name in df.loc[df.year!=year].INSTNM.unique():
        if fill_values.get(name):
            pass
        else:
            fill_values[name]={'LOCALE': np.NaN, 'CURROPER': np.NaN}
            
    print(len(fill_values))
    
#     df.LOCALE = df.INSTNM.map(lambda name: fill_values[name]['LOCALE'])
#     df.CURROPER = df.INSTNM.map(lambda name: fill_values[name]['CURROPER'])
    
    return df

In [147]:
df_2016_and_2017 = df.loc[(df.year==2017)|(df.year==2016)].copy()
df_2016_and_2017.head()

,INSTNM,HIGHDEG,CONTROL,REGION,LOCALE,CURROPER,TUITFTE,INEXPFTE,CDR3,LO_INC_DEBT_N,MD_INC_DEBT_N,HI_INC_DEBT_N,ICLEVEL,year
0,Alabama A & M University,4,1.0,5.0,NaN,NaN,9590.0,5888.0,0.189,1940,834,311,1.0,2016
1,University of Alabama at Birmingham,4,1.0,5.0,NaN,NaN,11007.0,17055.0,0.057,2763,1937,1602,1.0,2016
2,Amridge University,4,2.0,5.0,NaN,NaN,16512.0,5417.0,0.112,286,91,21,1.0,2016
3,University of Alabama in Huntsville,4,1.0,5.0,NaN,NaN,8902.0,10558.0,0.054,1115,764,747,1.0,2016
4,Alabama State University,4,1.0,5.0,NaN,NaN,8435.0,7889.0,0.194,2301,880,305,1.0,2016


In [136]:
df_2016_and_2017 = back_fill_from_year(df_2016_and_2017, 2017)

7177


In [148]:
df_2016_and_2017.loc[df.year!=2017].INSTNM.unique

ValueError: cannot reindex from a duplicate axis

In [140]:
df.loc[df.LOCALE==-3]

,INSTNM,HIGHDEG,CONTROL,REGION,LOCALE,CURROPER,TUITFTE,INEXPFTE,CDR3,LO_INC_DEBT_N,MD_INC_DEBT_N,HI_INC_DEBT_N,ICLEVEL,year
3950,College of Micronesia-FSM,2,1.0,9.0,-3.0,1.0,422.0,4227.0,NaN,NaN,NaN,NaN,2.0,2017
3951,Palau Community College,2,1.0,9.0,-3.0,1.0,5509.0,7911.0,NaN,NaN,NaN,NaN,2.0,2017
4240,College of the Marshall Islands,2,1.0,9.0,-3.0,1.0,810.0,3079.0,NaN,NaN,NaN,NaN,2.0,2017
6342,Saint Gregory The Great Seminary,0,2.0,4.0,-3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2017
6565,Georgie International Beauty Institute,1,3.0,3.0,-3.0,0.0,13819.0,13131.0,NaN,NaN,NaN,NaN,3.0,2017


In [99]:
df.loc[(df.year==2017)|(df.year==2016)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14233 entries, 0 to 7057
Data columns (total 14 columns):
INSTNM           14233 non-null object
HIGHDEG          14233 non-null int64
CONTROL          14233 non-null float64
REGION           14233 non-null float64
LOCALE           6614 non-null float64
CURROPER         7058 non-null float64
TUITFTE          13322 non-null float64
INEXPFTE         13323 non-null float64
CDR3             12307 non-null float64
LO_INC_DEBT_N    7156 non-null object
MD_INC_DEBT_N    7156 non-null object
HI_INC_DEBT_N    7156 non-null object
ICLEVEL          14233 non-null float64
year             14233 non-null int64
dtypes: float64(8), int64(2), object(4)
memory usage: 1.6+ MB


In [100]:
df_2016_and_2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14233 entries, 0 to 7057
Data columns (total 14 columns):
INSTNM           14233 non-null object
HIGHDEG          14233 non-null int64
CONTROL          14233 non-null float64
REGION           14233 non-null float64
LOCALE           13977 non-null object
CURROPER         13977 non-null object
TUITFTE          13322 non-null float64
INEXPFTE         13323 non-null float64
CDR3             12307 non-null float64
LO_INC_DEBT_N    7156 non-null object
MD_INC_DEBT_N    7156 non-null object
HI_INC_DEBT_N    7156 non-null object
ICLEVEL          14233 non-null float64
year             14233 non-null int64
dtypes: float64(6), int64(2), object(6)
memory usage: 1.6+ MB


In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33097 entries, 0 to 7169
Data columns (total 17 columns):
INSTNM                                     33097 non-null object
HIGHDEG                                    33097 non-null int64
CONTROL                                    33097 non-null float64
REGION                                     33096 non-null float64
LOCALE                                     0 non-null float64
CURROPER                                   0 non-null float64
TUITFTE                                    30904 non-null float64
INEXPFTE                                   30905 non-null float64
CDR3                                       33097 non-null float64
LO_INC_DEBT_N                              33097 non-null object
MD_INC_DEBT_N                              33097 non-null object
HI_INC_DEBT_N                              33097 non-null object
ICLEVEL                                    33097 non-null float64
year                                       33097

## Create function to Read in Data

In [171]:
def read_in_CollegeScorecard(columns):
    """
    Read in columns from files from inside the zip file. 
    Also assign a year to each DataFrame.
    
    Parameters:
    -----------
    columns: list,
        columns is a list of strings matching the desired column headers
        
    Returns:
    --------
    sheets: dictionary,
        sheets is a dictionary of year:DataFrame pairs
    """
    
    zip_file = ZipFile('CollegeScorecard_Raw_Data.zip')
    
    sheets = {}
    for year in range(1996, 2018):
        acyear = str(year)+'_'+str(year+1)[-2:]
        sheets[year] = pd.read_csv(zip_file.open('CollegeScorecard_Raw_Data/MERGED'+acyear+'_PP.csv'), usecols=columns)
        sheets[year]['YEAR'] = year
        sheets[year]['YEAR'] = pd.to_datetime(sheets[year]['YEAR'], format='%Y')
    return sheets

In [172]:
columns = ['INSTNM', 'HIGHDEG', 'CONTROL', 'REGION', 'LOCALE', 'LO_INC_DEBT_N', 'MD_INC_DEBT_N', 'HI_INC_DEBT_N',
           'LOAN_EVER', 'PELL_EVER', 'PCTPELL', 'ICLEVEL', 'CURROPER', 'TUITFTE', 'CDR3', 'INEXPFTE']
sheets = read_in_CollegeScorecard(columns)

## Create concatenated df of all sheets

In [217]:
def concatenate_all_sheets(sheets):
    """
    Concatenates DataFrames in a dictionary of DataFrames.
    
    Parameters:
    -----------
    sheets: dictionary,
        key value pairs are year and DataFrame associated to that year
        
    Returns:
    --------
    full_df: DataFrame
    """
    for year, df in sheets.items():
        df['iyear'] = df['YEAR']
        if year==1996:
            full_df = df.set_index([df.index, 'iyear'])
        else:
            full_df = pd.concat([full_df, df.set_index([df.index, 'iyear'])])
    return full_df

In [218]:
full_df = concatenate_all_sheets(sheets)
full_df.shape

(154228, 17)

## Clean huge DataFrame

In [219]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 154228 entries, (0, 1996-01-01 00:00:00) to (7057, 2017-01-01 00:00:00)
Data columns (total 17 columns):
INSTNM           154228 non-null object
HIGHDEG          154228 non-null int64
CONTROL          154205 non-null float64
REGION           154227 non-null float64
LOCALE           6614 non-null float64
CURROPER         7058 non-null float64
TUITFTE          133780 non-null float64
INEXPFTE         133775 non-null float64
PCTPELL          67384 non-null float64
CDR3             47815 non-null float64
LO_INC_DEBT_N    137486 non-null object
MD_INC_DEBT_N    137486 non-null object
HI_INC_DEBT_N    137486 non-null object
LOAN_EVER        137342 non-null object
PELL_EVER        137342 non-null object
ICLEVEL          154205 non-null float64
YEAR             154228 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(9), int64(1), object(6)
memory usage: 20.5+ MB


In [249]:
def back_fill_from_year(df, year):
    fill_values = {}
    for name in df.INSTNM.unique():
        fill_values[name] = {'LOCALE': df.loc[(df.INSTNM==name)].LOCALE.values[0], 
                             'CURROPER': df.loc[(df.INSTNM==name)].CURROPER.values[0]}
    
    for name in df.loc[df.YEAR!=year].INSTNM.unique():
        fill_values[name] = fill_values.get(name, {'LOCALE': np.NaN, 'CURROPER': np.NaN})
    
    df.LOCALE = df.INSTNM.map(lambda name: fill_values[name]['LOCALE'])
    df.CURROPER = df.INSTNM.map(lambda name: fill_values[name]['CURROPER'])
    
    return df

In [250]:
print(pd.concat([sheets[2017], sheets[2016]]).info())
back_fill_from_year(pd.concat([sheets[2017], sheets[2016]]), 2017).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14233 entries, 0 to 7174
Data columns (total 18 columns):
INSTNM           14233 non-null object
HIGHDEG          14233 non-null int64
CONTROL          14233 non-null int64
REGION           14233 non-null int64
LOCALE           6614 non-null float64
CURROPER         7058 non-null float64
TUITFTE          13322 non-null float64
INEXPFTE         13323 non-null float64
PCTPELL          12718 non-null float64
CDR3             12307 non-null float64
LO_INC_DEBT_N    7156 non-null object
MD_INC_DEBT_N    7156 non-null object
HI_INC_DEBT_N    7156 non-null object
LOAN_EVER        6976 non-null object
PELL_EVER        6976 non-null object
ICLEVEL          14233 non-null int64
YEAR             14233 non-null datetime64[ns]
iyear            14233 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(6), int64(4), object(6)
memory usage: 2.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 14233 entries, 0 to 7174
Data columns (tota